# Dutch Corpora

### Instal tempo-embeddings

In [22]:
# Install tempo-embeddings from GitHub
# This can also refer to a specific version or branch
# %pip install ..

%pip install --upgrade pip  # Required for properly resolving dependencies
%pip uninstall -y tempo_embeddings  # Remove existing installation
%pip install --upgrade git+https://github.com/Semantics-of-Sustainability/tempo-embeddings.git


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Processing /Users/jose/Repos/tempo-embeddings
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tempo_embeddings: filename=tempo_embeddings-0.0.1-py3-none-any.whl size=36869 sha256=502c884b42f6c255f2e7bc8771044f364e6265e71e423b7841c30165b63a7161
  Stored in directory: /Users/jose/Library/Caches/pip/wheels/de/25/96/d92b7a130b730e0ab67770d76841f36cb3d1f9cda32a4a539b
Successfully built tempo_embeddings
  Attempting uninstall: tempo_embeddings
    Found existing installation: tempo_embeddings 0.0.1
    Uninstalling tempo_embeddings-0.0.1:
      Successfully uninstalled

### Import tempo-embeddings and start logging

In [23]:
# make sure installation has succeeded
import tempo_embeddings

from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Optional: Load NLP Pipeline

It is possible to use the Stanza library to pre-tokenize and split the text into sentences. This process is not perfect (it tends to overfragment the text if it has too many rare characters) and takes some extra amount of time and memory resources. If `nlp_pipeline = None` then a naive tokenization is done by splitting with spaces and stripping punctuation characters from the text.

In [24]:
%pip install stanza==1.7.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [25]:
nlp_pipeline = None

# import stanza
# stanza.download('nl')
# nlp_pipeline = stanza.Pipeline("nl", processors='tokenize')

## Load Data

The data needs to be downloaded and provided in the path configured in the next cell.

NOTE: You have to manually adapt the `DATA_DIR` below.

In [26]:
%autoreload now

import operator
from functools import reduce
from pathlib import Path
from tqdm import tqdm
from tempo_embeddings.text.corpus import Corpus

In [27]:
WINDOW_SIZE = 300 # Size of passages in characters
USE_FULL_SENTENCES = False # For now, this parameter overrides the window size. Instead of splitting arbitrarily the passages in WINDO_SIZE chunks, each Sentence will be a Passage

RANDOM_SAMPLE_ANP = 200
RANDOM_SAMPLE_STATEN_GENERAAL = 200

STATEN_GENERAAL_BLACKLIST = ["1987"]

FILTER_TERMS = ["duurzaam"]  # Search term(s) for filtering the corpus. If empty, it will only create Passages for the pieces of text that match the FILTER_TERMS

In [28]:
## NOTE: Adapt the `DATA_DIR` below manually!
## For a shared Google Drive, create a shortcut into your own Google Drive
## See https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab

try:
    import google.colab
    IN_COLAB = True
except ModuleNotFoundError:
    IN_COLAB = False

LOCAL_PATHS: list[Path] = [
    Path.home() / "Documents" / "SemanticsOfSustainability" / "data" / "Joris",
    Path.home() / "SEED_DATA" / "SemanticsSustainability", # local angel
    Path("/data/volume_2/data"),  # Research Cloud
    Path("/data/storage-semantics-of-sustainability/data/"), # New Research Cloud
    Path("/home/cschnober/data/"),  # Snellius
]

if IN_COLAB:
    from google.colab import drive

    drive.mount("/content/drive")

    DATA_DIR = Path("/content/drive/MyDrive/Data/")
else:
    try:
        DATA_DIR = next(path for path in LOCAL_PATHS if path.is_dir())
    except StopIteration as e:
        raise DirectoryNotFoundError(f"Data directory not found.") from e

assert DATA_DIR.is_dir(), f"Data dir '{DATA_DIR}' not found."

### ANP

In [29]:
ANP_DIR = DATA_DIR / "ANP"
assert RANDOM_SAMPLE_ANP == 0 or ANP_DIR.is_dir(), f"{ANP_DIR} not found."

In [30]:
import random


random.seed(0)

anp_files = list(ANP_DIR.glob("ANP_????.csv.gz"))

if RANDOM_SAMPLE_ANP and len(anp_files) > RANDOM_SAMPLE_ANP:
    anp_files = random.sample(
        list(ANP_DIR.glob("ANP_????.csv.gz")), k=RANDOM_SAMPLE_ANP
    )

print(f"Found {len(anp_files)} ANP Files")
anp_files[:10]

Found 2 ANP Files


[PosixPath('/Users/jose/SEED_DATA/SemanticsSustainability/ANP/ANP_1983.csv.gz'),
 PosixPath('/Users/jose/SEED_DATA/SemanticsSustainability/ANP/ANP_1984.csv.gz')]

In [31]:
anp_corpus = (
    reduce(
        operator.add,
        (
            Corpus.from_csv_file(
                path,
                filter_terms=FILTER_TERMS,
                text_columns=["content"],
                encoding="iso8859_15",
                compression="gzip",
                delimiter=";",
                window_size=WINDOW_SIZE,
                nlp_pipeline=nlp_pipeline
            )
            for path in tqdm(anp_files, unit="file")
        ),
    )
    if anp_files
    else Corpus()
)

len(anp_corpus)

100%|██████████| 2/2 [00:01<00:00,  1.36file/s]


54

### Staten Generaal

In [32]:
STATEN_GENERAAL_DIR = DATA_DIR / "StatenGeneraal"

assert RANDOM_SAMPLE_STATEN_GENERAAL == 0 or STATEN_GENERAAL_DIR.is_dir()

In [33]:
glob195x = "StatenGeneraal_19[0-9]?.csv.gz"  # Pattern for files from 1950-1999
glob20xx = "StatenGeneraal_2???.csv.gz"  # Pattern for files from 2000

files_195x = list(STATEN_GENERAAL_DIR.glob(glob195x))
files_20xx = list(STATEN_GENERAAL_DIR.glob(glob20xx))

sg_files = [
    file
    # Merge files from patterns
    for file in files_20xx + files_195x
    # Remove blacklisted files:
    for blacklisted in STATEN_GENERAAL_BLACKLIST
    if blacklisted not in file.name
]

if RANDOM_SAMPLE_STATEN_GENERAAL and RANDOM_SAMPLE_STATEN_GENERAAL < len(sg_files):
    sg_files = random.sample(sg_files, k=RANDOM_SAMPLE_STATEN_GENERAAL)

print(f"Found {len(sg_files)} STAATEN_G Files")
sorted(sg_files[:10])

Found 2 STAATEN_G Files


[PosixPath('/Users/jose/SEED_DATA/SemanticsSustainability/StatenGeneraal/StatenGeneraal_2017.csv.gz'),
 PosixPath('/Users/jose/SEED_DATA/SemanticsSustainability/StatenGeneraal/StatenGeneraal_2018.csv.gz')]

In [34]:
%autoreload now

import csv

csv.field_size_limit(100000000)

sg_corpus = (
    reduce(
        operator.add,
        (
            Corpus.from_csv_file(
                path,
                filter_terms=FILTER_TERMS,
                text_columns=["Content"],
                encoding="utf-8",
                compression="gzip",
                delimiter=";",
                window_size=WINDOW_SIZE,
                nlp_pipeline=nlp_pipeline
            )
            for path in tqdm(sg_files, unit="file")
        ),
    )
    if sg_files
    else Corpus()
)

len(sg_corpus)

100%|██████████| 2/2 [00:01<00:00,  1.07file/s]


1504

### Show the Passages that were created in the corpus

In [35]:
for p in sg_corpus.passages[:20]:
    print(len(p), p)

303 Passage('economische groei consequent milieuvernietiging op, in meerdere of mindere mate. Economische groei zou "duurzaam" te noemen zijn als die wordt gekoppeld aan technologische vooruitgang en toenemende efficiëntie. Zo zouden we de taart kunnen vergroten en alle milieuproblematiek de baas kunnen worden. Een', {'': '204', 'RecId': 'h-ek-20162017-14-3', 'chamber': 'EersteKamer', 'date': '2017-01-17', 'speakers': 'De voorzitter :|Mevrouw Teunissen ( PvdD ):|Mevrouw Vos ( GroenLinks ):|De heer Van Hattem ( PVV ):|Mevrouw Stienen ( D66 ):|De heer Verheijen ( PvdA ):|Mevrouw Meijer ( SP ):|De voorzitter :|Minister Schultz van Haegen-Maas Geesteranus :|Mevrouw Vos ( GroenLinks ):|Minister Schultz van Haegen-Maas Geesteranus :|Mevrouw Vos ( GroenLinks ):|Minister Schultz van Haegen-Maas Geesteranus :|De heer Verheijen ( PvdA ):|Minister Schultz van Haegen-Maas Geesteranus :|De heer Verheijen ( PvdA ):|Minister Schultz van Haegen-Maas Geesteranus :|De heer Van Hattem ( PVV ):|Minister S

### Merge

In [36]:
corpus = anp_corpus # + sg_corpus
len(corpus)

54

## Load Model

In [37]:
%autoreload now
from tempo_embeddings.embeddings.model import EmbeddingsMethod
from tempo_embeddings.embeddings.model import  SentenceTransformerModelWrapper

kwargs = {"accelerate": True}

### Sentence Transformers
MODEL_NAME = "NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers"
# MODEL_NAME = "textgain/allnli-GroNLP-bert-base-dutch-cased"
model_class = SentenceTransformerModelWrapper

model = model_class.from_pretrained(MODEL_NAME, **kwargs)
model.embeddings_method = EmbeddingsMethod.MEAN
model

## Create or Open existing Database

In [38]:
import os
from tempo_embeddings.embeddings.weaviate_database import WeaviateDatabaseManager

# from dotenv import load_dotenv
# load_dotenv()
# MODEL_NAME = "NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers"
# EMBED_CONFIG = {"type":"hf", "api_key": os.getenv('HUGGING_FACE_API_KEY')}

EMBED_CONFIG = {"type": "default", "model": model} 
db = WeaviateDatabaseManager(embedder_name=MODEL_NAME,embedder_config=EMBED_CONFIG)
db.connect()

collection_name = "AnpSgCorpus"

01:36:33 INFO:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
01:36:33 INFO:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
01:36:33 INFO:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
01:36:33 INFO:<class 'weaviate.client.WeaviateClient'>
01:36:33 INFO:HTTP Request: GET http://localhost:8080/v1/.well-known/ready "HTTP/1.1 200 OK"
01:36:33 INFO:Weaviate Server Is Up: True
01:36:33 INFO:HTTP Request: GET http://localhost:8080/v1/.well-known/ready "HTTP/1.1 200 OK"


In [39]:
%autoreload now

## Compute & Save Embeddings (If DB didn't existed)

In [40]:
try:
    db.create_new_collection(collection_name, corpus)
except ValueError:
    print(f"Collection '{collection_name}' already exists and has {db.get_collection_count(collection_name)} records")
    

01:36:33 INFO:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
01:36:33 INFO:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
01:36:33 INFO:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
Embeddings:  86%|████████▌ | 6/7 [00:07<00:01,  1.22s/batch]
01:36:40 INFO:Added 54 new documents.
01:36:40 INFO:Created NEW collection 'AnpSgCorpus'


## TEST: Retrieve Records from Database

In [41]:
%autoreload now

corpus_mini = db.get_corpus(collection_name, filter_words=["duurzaam"])
print(len(corpus_mini))

01:36:41 INFO:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
01:36:41 INFO:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
01:36:41 INFO:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"


98


In [42]:
if len(corpus_mini):
    db.create_new_collection("AnpDuurzaam", corpus_mini)

01:36:41 INFO:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
01:36:41 INFO:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
01:36:41 INFO:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
Embeddings:  92%|█████████▏| 12/13 [00:04<00:00,  2.84batch/s]
01:36:45 INFO:Added 98 new documents.
01:36:45 INFO:Created NEW collection 'AnpDuurzaam'
